In [1]:
import sys

import pandas as pd
import numpy as np
import datetime

from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

# from cyclic_boosting import binning, flags, CBPoissonRegressor, observers, common_smoothers
# from cyclic_boosting.smoothing.onedim import SeasonalSmoother, IsotonicRegressor
# from cyclic_boosting.plots import plot_analysis
# from cyclic_boosting.pipelines import pipeline_CBPoissonRegressor

import timeseries_plots as tsp
import kpi_calculation as kpi
from sklearn.preprocessing import LabelEncoder
%matplotlib inline 

# from prophet import Prophet

# Load data

In [2]:
df= pd.read_csv('forecasting_cleaned.csv',parse_dates=['DATE'])

In [3]:
df.sample(5)

,DATE,PRICE,SPACE,SPACE_binned,CATEGORIES,median_PRICE_CATEGORY_SPACE_binned_DATE,CATEGORY_N_DATES
10512,2019-05-29,1500.0,94.0,"(50, 100]","APARTMENT, DUPLEX, MAISONETTE",1500.0,451
2657,2018-05-16,3200.0,120.0,"(100, 150]","APARTMENT, ATTIC_FLAT",2650.0,305
4271,2018-07-28,1920.0,96.0,"(50, 100]","APARTMENT, FLAT",2210.0,1370
7423,2018-12-18,1100.0,20.0,"(0, 50]","APARTMENT, STUDIO",1100.0,224
24651,2021-05-26,1240.0,51.0,"(50, 100]","APARTMENT, FLAT",1395.0,1370


# Baseline

From EDA we know that median price per category in space bins does not change a lot with time. Let's use it as baseline

In [4]:
df['total'] = 'Total'
df['C_DATE'] = df['DATE']

In [5]:
df_base = df[df['C_DATE']<='2020-12-31'].groupby(['CATEGORIES','SPACE_binned']).agg({'median_PRICE_CATEGORY_SPACE_binned_DATE':'median'}).reset_index()
df_base.rename(columns={'median_PRICE_CATEGORY_SPACE_binned_DATE':'BASELINE'}, inplace=True)
df = df.merge(df_base,on=['CATEGORIES','SPACE_binned'],how='left')

In [6]:
df[['DATE','PRICE','BASELINE']].sample(10)

,DATE,PRICE,BASELINE
8283,2019-02-04,1750.0,1575.0
4638,2018-08-16,2640.0,2390.0
20779,2020-11-13,650.0,855.0
13360,2019-10-28,1377.0,1400.0
11683,2019-07-31,1950.0,1400.0
20950,2020-11-20,1240.0,1400.0
11453,2019-07-18,1280.0,1400.0
297,2018-01-17,1440.0,1400.0
28443,2021-12-03,2180.0,1400.0
1591,2018-03-22,1070.0,855.0


In [7]:
tsp.plot_weekly_per_agg(df,actuals_col='PRICE',var_cols=['BASELINE'], agg_col='total', 
                           agg='median', limit_n_plots=3, title_text='',
                          date_from='2021-01-01',date_upto='2021-12-31',plot_two_years_ago=False,plot_last_year=True)

alt.ConcatChart(...)

#### As expected the baseline based on median is a bit below average prices

In [8]:
tsp.plot_weekly_per_agg(df,actuals_col='PRICE',var_cols=['BASELINE'], agg_col='CATEGORIES', 
                           agg='median', limit_n_plots=3, title_text='',
                          date_from='2021-01-01',date_upto='2021-12-31',plot_two_years_ago=True,plot_last_year=True)

alt.ConcatChart(...)

alt.ConcatChart(...)

# ML model

## Data enchantments

In [9]:
df = tsp.enrich_day(df,time_col='DATE')

In [10]:
df_agg_prices = df.groupby(['CATEGORIES','SPACE_binned','YEAR']).agg({'median_PRICE_CATEGORY_SPACE_binned_DATE':'median'}).reset_index()
df_agg_prices['next_YEAR'] = df_agg_prices['YEAR']+1
df_agg_prices.drop(columns=['YEAR'],inplace=True)
df_agg_prices = df_agg_prices.rename(columns={'median_PRICE_CATEGORY_SPACE_binned_DATE':'median_PRICE_CATEGORY_SPACE_binned_last_YEAR','next_YEAR':'YEAR'})

In [11]:
df = df.merge(df_agg_prices, on=['CATEGORIES','SPACE_binned','YEAR'], how='left')
df.head(3).T

,0,1,2
DATE,2018-01-01 00:00:00,2018-01-02 00:00:00,2018-01-02 00:00:00
PRICE,3000.0,1510.0,980.0
SPACE,70.0,61.0,54.0
SPACE_binned,"(50, 100]","(50, 100]","(50, 100]"
CATEGORIES,"APARTMENT, FURNISHED_FLAT","APARTMENT, FLAT","APARTMENT, FLAT"
median_PRICE_CATEGORY_SPACE_binned_DATE,3000.0,1430.0,1430.0
CATEGORY_N_DATES,996,1370,1370
total,Total,Total,Total
C_DATE,2018-01-01 00:00:00,2018-01-02 00:00:00,2018-01-02 00:00:00
BASELINE,2250.0,1400.0,1400.0


In [12]:
# Creating a instance of label Encoder.
le = LabelEncoder()
# Using .fit_transform function to fit label
# encoder and return encoded label
df['CATEGORIES_ID'] = le.fit_transform(df['CATEGORIES'])

## Modeling

In [13]:
df_train = df.loc[df['DATE']<='2020-12-31']
df_test = df.loc[df['DATE']>'2020-12-31']

In [14]:
y_train = np.asarray(df_train['PRICE'])
y_test = np.asarray(df_test['PRICE'])
obs_cols = ['PRICE','THIS_WEEK_MONDAY','BASELINE','median_PRICE_CATEGORY_SPACE_binned_DATE',
            'SPACE_binned','CATEGORY_N_DATES','CATEGORIES','DATE','C_DATE','WEEKDAY_NAME','total']
X_train = df_train.drop(columns=obs_cols)
X_test = df_test.drop(columns=obs_cols)
X_test_org = X_test.copy()

In [15]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
# Setup XGBoost hyperparameter grid
xgb_param_grid = {"learning_rate": [0.05, 0.1, 0.2], # Step size shrinkage used in update to prevents overfitting.
                  "max_depth"        : [6, 8, 9, 10], # Maximum depth of a tree. 
                  "min_child_weight" : [1, 3, 5, 7], # Minimum number of instances required in a child node
                  "gamma"            : [0.0, 0.1, 0.2, 0.3], # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                  "colsample_bytree" : [0.3, 0.4, 0.6, 0.8] # Number of features supplied to a tree
}

# # Setup XGBoost hyperparameter grid
# xgb_param_grid = {"learning_rate": [0.05], # Step size shrinkage used in update to prevents overfitting.
#                   "max_depth"        : [6], # Maximum depth of a tree. 
#                   "min_child_weight" : [1], # Minimum number of instances required in a child node
#                   "gamma"            : [0.0, 0.1], # Minimum loss reduction required to make a further partition on a leaf node of the tree.
#                   "colsample_bytree" : [0.3] # Number of features supplied to a tree
# }

# Instantiate XGBoost regressor
xgb_reg = xgb.XGBRegressor(seed = 42, objective ='reg:absoluteerror')

# Setup GridSearchCV
xgb_reg_grid = GridSearchCV(estimator = xgb_reg, param_grid = xgb_param_grid, cv=5, n_jobs=-1, verbose=0) 

# Fit Grid Search on training dataset
xgb_reg_grid.fit(X_train, y_train)

# Get best model (based on best parameters) and predict on test set
best_xgb_reg = xgb_reg_grid.best_estimator_
y_pred_test = best_xgb_reg.predict(X_test)

## KPIs

In [16]:
df_test['ML FORECAST'] = y_pred_test

In [17]:
df_evl = df.copy()
df_evl = df_evl.merge(df_test[['DATE','SPACE','CATEGORIES','ML FORECAST']],on=['DATE','SPACE','CATEGORIES',],how='left')
df_evl['total'] = 'Total'
df_evl['C_DATE'] = df_evl['DATE']

In [18]:
tsp.plot_weekly_per_agg(df_evl,actuals_col='PRICE',var_cols=['BASELINE','ML FORECAST'], agg_col='total', 
                           agg='median', limit_n_plots=3, title_text='',
                          date_from='2021-01-01',date_upto='2021-12-31',plot_two_years_ago=False,plot_last_year=True)

alt.ConcatChart(...)

In [25]:
tsp.plot_weekly_per_agg(df_evl,actuals_col='PRICE',var_cols=['BASELINE','ML FORECAST'], agg_col='total', 
                           agg='max', limit_n_plots=3, title_text='',
                          date_from='2021-01-01',date_upto='2021-12-31',plot_two_years_ago=False,plot_last_year=True)

alt.ConcatChart(...)

In [24]:
tsp.plot_weekly_per_agg(df_evl,actuals_col='PRICE',var_cols=['BASELINE','ML FORECAST'], agg_col='total', 
                           agg='min', limit_n_plots=3, title_text='',
                          date_from='2021-01-01',date_upto='2021-12-31',plot_two_years_ago=False,plot_last_year=True)

alt.ConcatChart(...)

In [19]:
df_evl[df_evl['C_DATE']>df_test['DATE'].min()][['DATE','SPACE','CATEGORIES','PRICE','ML FORECAST','BASELINE']].sample(3)

,DATE,SPACE,CATEGORIES,PRICE,ML FORECAST,BASELINE
30681,2021-12-17,63.0,"APARTMENT, FLAT",1390.0,1267.189941,1400.0
29576,2021-11-03,57.0,"APARTMENT, FLAT",1100.0,1178.774780,1400.0
25220,2021-05-11,120.0,"APARTMENT, FLAT",2270.0,2230.418945,2390.0


In [26]:
kpi.KPI_per_agg_var(df_test,var='total',limit=5,target_name='PRICE',pred_name='ML FORECAST',pred_name_comp='BASELINE',product_col='CATEGORIES',
                label='Total | MLForecast | Week-Category',granularity_list=['WEEK','CATEGORIES'],
                    pred_from=df_test['DATE'].min().date(),pred_upto=df_test['DATE'].max().date())

,Total Actuals,Total Predictions,% predictions for 0 actuals,Categories,Mean Actuals,Mean Predictions,MD,Bias,RMAD,RMSE
total,,,,,,,,,,
ML FORECAST Total,"11,655,884","11,307,295",0.0%,19,"21,827.50","21,174.71",-652.79,-0.03,6.5%,"3,781.26"
BASELINE Total,"11,655,884","11,222,208",0.0%,19,"21,827.50","21,015.37",-812.13,-0.04,7.7%,"4,215.51"


In [28]:
kpi.KPI_per_agg_var(df_test,var='CATEGORIES',limit=3,target_name='PRICE',pred_name='ML FORECAST',pred_name_comp='BASELINE',product_col='CATEGORIES',
                label='Total | ML Forecast | Week-Category',granularity_list=['WEEK','CATEGORIES'],
                    pred_from=df_test['DATE'].min().date(),pred_upto=df_test['DATE'].max().date())

,Total Actuals,Total Predictions,% predictions for 0 actuals,Categories,Mean Actuals,Mean Predictions,MD,Bias,RMAD,RMSE
CATEGORIES,,,,,,,,,,
"ML FORECAST APARTMENT, FLAT","8,465,579","8,273,173",0.0%,1,"159,727.91","156,097.61","-3,630.31",-0.02,4.1%,"9,919.73"
"BASELINE APARTMENT, FLAT","8,465,579","8,172,150",0.0%,1,"159,727.91","154,191.51","-5,536.40",-0.03,4.9%,"10,412.86"
"ML FORECAST APARTMENT, FURNISHED_FLAT","1,149,668","1,178,017",0.0%,1,"21,691.85","22,226.73",534.89,0.02,7.4%,"2,061.25"
"BASELINE APARTMENT, FURNISHED_FLAT","1,149,668","1,207,660",0.0%,1,"21,691.85","22,786.04","1,094.19",0.05,8.1%,"2,383.06"
"ML FORECAST APARTMENT, ROOF_FLAT","489,494","460,441",0.0%,1,"9,413.35","8,854.63",-558.72,-0.06,9.7%,"1,278.22"
"BASELINE APARTMENT, ROOF_FLAT","489,494","476,850",0.0%,1,"9,413.35","9,170.19",-243.15,-0.03,11.0%,"1,466.38"


In [22]:
tsp.plot_weekly_per_agg(df_evl,actuals_col='PRICE',var_cols=['BASELINE','ML FORECAST'], agg_col='CATEGORIES', 
                           agg='mean', limit_n_plots=10, title_text='',
                          date_from='2021-01-01',date_upto='2021-12-31',plot_two_years_ago=False,plot_last_year=True)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)

alt.ConcatChart(...)